# Working with text files
Interact with CSV
## Learn
```julia
read("auto.txt", String)     # read file into a string
replace(raw_str, '\t'=>' ')  # replace char in a string
IOBuffer(str_no_tab)         # IO from a string
DataFrame(:abc => Float64[]) # Create with empty column
Float64[]                    # alias for Array{Float64, 1}
allowmissing!()
# isqual() vs ==
something()
tryparse()
ByRow()
count()
eachcol()
Iterators.flatten()
```

In [ ]:
using DataFrames
using CSV
using Arrow
using Statistics

using Test

Download the data set

In [ ]:
download("https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data-original",
         "auto.txt")

Use `readlines` to read the file.

In [ ]:
readlines("auto.txt")

Notice the following
1. no header
2. some tab, some space separated
3. "NA" for missing value

Different options to parse into a `DataFrame`.

### Replace tabs with spaces
Read file into a string

In [ ]:
raw_str = read("auto.txt", String)

Replace tab by space
This will not work if column text has tab in it.

In [ ]:
str_no_tab = replace(raw_str, '\t'=>' ')

Create `IOBuffer` from the string

In [ ]:
io = IOBuffer(str_no_tab)

Read data from `io` into CSV, and notice the following
1. delimiter is space
2. ignore consecutive delimiter
3. specify column header
4. specify missing value is represented by "NA"

Notice a pipe is used to pass output from `CSV` to `DataFrame`.

In [ ]:
df1 = CSV.File(
    io,
    delim = ' ',
    ignorerepeated = true,
    header = [
        :mpg, :cylinders, :displacement, :horsepower,
        :weight, :acceleration, :year, :origin, :name,  
    ],
    missingstring = "NA",
) |> DataFrame

Control the amount of data displayed

In [ ]:
ENV["COLUMNS"] = 200
ENV["LINES"] = 15
df1

### Read directly

In [ ]:
df_raw = CSV.File("auto.txt", header=[:metrics, :name]) |> DataFrame

Split `:metrics` manually

In [ ]:
str_metrics = split.(df_raw[!,:metrics])

Create empty `DataFrame` with list comprehension.  Notice `Float64[]` is an alias for `Array{Float64, 1}`.

In [ ]:
df1_2 = DataFrame([col => Float64[] for col in [:mpg, :cylinders, :displacement, :horsepower, :weight, :acceleration, :year, :origin]])

Allow missing values

In [ ]:
allowmissing!(df1_2, [:mpg, :horsepower])

Note that `Float64?` indicates allowing missing values.
Populate data frame

In [ ]:
for row in str_metrics
    push!(df1_2, [v == "NA" ? missing : parse(Float64,v) for v in row])
end

df1_2

Add `:name` from `df_raw`

In [ ]:
df1_2[!,:name] = df_raw[!,:name] # copy by reference
@test df1_2.name === df_raw.name

df1_2[!,:name] = df_raw[:,:name] # copy by value
@test df1_2.name !== df_raw.name

@test isequal(df1_2, df1) # different from ==
@test ismissing(df1_2 == df1)

df1_2

### Read with one liner 

In [ ]:
df1_3 = select(df_raw,
    :metrics => ByRow(
       x -> something.(tryparse.(Float64, split(x)), missing)
    ) => [:mpg, :cylinders, :displacement, :horsepower, :weight, :acceleration, :year, :origin],
    :name
)
@test isequal(df1_3, df1)

Count the number of missing values

In [ ]:
Ans = [count(ismissing, col) for col in eachcol(df1)]
sum(Ans)

Use `mapcols` to the number of missings per column

In [ ]:
Ans = mapcols(col -> count(ismissing, col), df1)
@test Ans isa DataFrame
Ans

Find rows with missing values

In [ ]:
Ans = filter(row -> any(ismissing, row), df1)

Create a brand column from the name column

In [ ]:
Ans = first.(split.(df1[!,:name]))
df1[!,:brand] = Ans
df1

In [ ]:
# Drop rows with missing
df2 = dropmissing(df1)

Find rows that is "saab" brand by indexing

In [ ]:
df2[df2[!,:brand] .== "saab", :]

Find rows that is "saab" brand by filtering

In [ ]:
filter("brand" => ==("saab"), df2)

Save to files

In [ ]:
CSV.write("auto2.csv", df2)
Arrow.write("auto2.arrow", df2)

In [ ]:
# inspect the output file
readlines("auto2.csv")